In [ ]:
# [NOTEBOOK TO ADD TO DATABASE] -------------------
# [INCLUDES] ------------------------------------
from Includes import *
import yfinance
import sqlite3
import pandas
import random, time

SYMBOLS = [
    'GOLDBEES.NS',   # Gold ETF (India)
    'SILVERETF.NS',  # Silver ETF (India)
    'GC=F',          # Gold futures (global)
    'SI=F'           # Silver futures (global)
]


TABLE_NAMES = [
    'goldbees_india',
    'silveretf_india',
    'gold_global',
    'silver_global'
]


for entries, table in zip(SYMBOLS, TABLE_NAMES):
        
    # [CREATE TABLE] ----------------------------- 
    TABLE_NAME = table
    conn = sqlite3.connect("databases/indices.db")
    cur = conn.cursor()

    # [GET DATA] -----------------------------
    time.sleep(random.uniform(0.5, 1.1))
    TICKER = entries
    PERIOD = "max"
    def _Fetch():
        ticker_data = yfinance.download(tickers=TICKER, period=PERIOD, multi_level_index=False)
        if ticker_data is None or ticker_data.empty:
            print("[FETCH] Empty ticker data returned")
            continue
        else:
            # print(ticker_data.head())
            return ticker_data
    ticker_data = _Fetch()

    # [FORMAT DATA] -----------------------------
    # ticker_data.drop(columns=['Volume'], inplace=True)
    ticker_data.reset_index(inplace=True) # remove the date from index
    ticker_data.rename(columns={'Date' : 'date', 'Close' : 'close', 'High' : 'high', 'Low' : 'low', 'Open' : 'open', 'Volume' : 'volume'}, inplace=True)
    print(ticker_data)

    # [SAVE TO DATABASES' TABLE] -----------------------------
    ticker_data.to_sql(table, conn, if_exists='replace', index=False)
    df = pandas.read_sql(f"SELECT * FROM {table}", conn)

    # [PLOT FOR VERIFICATION] -----------------------------
    import matplotlib.pyplot as plt

    def plotter(conn, table):
        df = pandas.read_sql(f"SELECT * FROM {table}", conn)
        df['date'] = pandas.to_datetime(df['date']) # IMPORTANT
        df.set_index('date', inplace=True)

        plt.plot(df['close'])
        plt.show()

    plotter(conn, table)

conn.close()

In [2]:
import sqlite3

conn = sqlite3.connect("databases/indices.db")
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [t[0] for t in cur.fetchall()]

print("Tables:", tables)


Tables: ['nifty50', 'nifty100', 'nifty200', 'nifty500', 'midcap100', 'midcap50', 'smallcap100', 'smallcap50', 'banknifty', 'psu_bank', 'private_bank', 'it', 'auto', 'metal', 'pharma', 'fmcg', 'energy', 'realty', 'infra', 'finance', 'media', 'indiavix', 'etf_bankbees', 'etf_psubankbees', 'etf_itbees', 'etf_pharmabees', 'etf_cpse', 'etf_midcap', 'etf_smallcap', 'etf_niftybees', 'etf_juniorbees', 'etf_infrabees', 'etf_energy', 'goldbees_india', 'silveretf_india', 'gold_global', 'silver_global']


In [3]:
bad_tables = []

for table in tables:
    print(f"\nChecking {table}...")
    try:
        df = pandas.read_sql(f"SELECT * FROM {table}", conn)
        
        # CASE 1 — table has zero rows
        if df.empty:
            print(f"[EMPTY] {table} has no data.")
            bad_tables.append(table)
            continue
        
        # CASE 2 — table has only 1 row (invalid ticker)
        if len(df) <= 1:
            print(f"[BAD] {table} has only {len(df)} row(s).")
            bad_tables.append(table)
            continue
        
        # CASE 3 — date column is missing or broken
        if 'date' not in df.columns:
            print(f"[BAD] {table} has no 'date' column.")
            bad_tables.append(table)
            continue
        
        # CASE 4 — all dates identical → bad ticker
        if df['date'].nunique() == 1:
            print(f"[BAD] {table} has only one unique date.")
            bad_tables.append(table)
            continue
        
        print(f"[OK] {table} seems valid.")

    except Exception as e:
        print(f"[ERROR] {table}: {e}")
        bad_tables.append(table)



Checking nifty50...
[OK] nifty50 seems valid.

Checking nifty100...
[OK] nifty100 seems valid.

Checking nifty200...
[OK] nifty200 seems valid.

Checking nifty500...
[OK] nifty500 seems valid.

Checking midcap100...
[EMPTY] midcap100 has no data.

Checking midcap50...
[OK] midcap50 seems valid.

Checking smallcap100...
[EMPTY] smallcap100 has no data.

Checking smallcap50...
[EMPTY] smallcap50 has no data.

Checking banknifty...
[OK] banknifty seems valid.

Checking psu_bank...
[OK] psu_bank seems valid.

Checking private_bank...
[EMPTY] private_bank has no data.

Checking it...
[OK] it seems valid.

Checking auto...
[OK] auto seems valid.

Checking metal...
[OK] metal seems valid.

Checking pharma...
[OK] pharma seems valid.

Checking fmcg...
[OK] fmcg seems valid.

Checking energy...
[OK] energy seems valid.

Checking realty...
[OK] realty seems valid.

Checking infra...
[OK] infra seems valid.

Checking finance...
[EMPTY] finance has no data.

Checking media...
[OK] media seems val

In [4]:
for table in bad_tables:
    print(f"Dropping table {table}...")
    cur.execute(f"DROP TABLE IF EXISTS {table}")
    conn.commit()


Dropping table midcap100...
Dropping table smallcap100...
Dropping table smallcap50...
Dropping table private_bank...
Dropping table finance...
Dropping table etf_smallcap...
